This note book will show you how does the Teams-ai echo bot server work.
 
First, we need to import related packages

In [ ]:
# Import necessary libraries
from botbuilder.schema import Activity
from flask import Flask, request, jsonify
from botbuilder.core import BotFrameworkAdapterSettings, TurnContext
from teams import Application, ApplicationOptions, TurnState
import sys
import traceback
import os


> You can find the following code in `config.py`

Then we can initialize three constants. **Config** is from config.py, containing `port`, `app_id` and `app_password`.

In [ ]:
# Create the bot application
class Config:
    """Bot Configuration"""
    port = 3978
    app_id = os.environ.get("BOT_ID", "")
    app_password = os.environ.get("BOT_PASSWORD", "")

> You can find the following code in `bot.py`, and the codes do the following:

1. We will define a bot **app** using a `BotFrameworkAdapterSettings` with **config.app_id** and **config.app_password** as a `BotFrameworkAdapterSettings` auth setting.
2. Then we can define `on_message()` message handler and `on_error()` error handler of **app**.

In [17]:

config = Config()
app = Application[TurnState](
    ApplicationOptions(
        bot_app_id=config.app_id,
        auth=BotFrameworkAdapterSettings(
            app_id=config.app_id,
            app_password=config.app_password,
        ),
    )
)


@app.activity("message")
async def on_message(context: TurnContext, _state: TurnState):
    await context.send_activity(f"you said: {context.activity.text}")
    return True


@app.error
async def on_error(context: TurnContext, error: Exception):
    # This check writes out errors to console log .vs. app insights.
    # NOTE: In production environment, you should consider logging this to Azure
    #       application insights.
    print(f"\n [on_turn_error] unhandled error: {error}", file=sys.stderr)
    traceback.print_exc()

    # Send a message to the user
    await context.send_activity("The bot encountered an error or bug.")

## Test Case ##

The provided test case is testing the `on_message()` function. The purpose of this test case is to verify the behavior of the `on_message()` function when it receives a user message. It ensures that the function correctly processes the message and sends a response activity with the appropriate content.

If the `on_message()` works correctly, it will prints the string `"you said: Hello, test message!"`. You can change the words of `context.activity.text` to see the result.

In [ ]:

## Test the bot activity on_message()

from botbuilder.core import TurnContext
from unittest.mock import MagicMock, patch

def mock_send_activity(message):
    print("mocked: ", message)

async def test_on_message():
    # Mock the content of the context object
    context = MagicMock(spec=TurnContext)
    context.activity.channel_id = "mock_channel"
    context.activity.from_property.id = "mock_user"

    # Set the activity properties
    context.activity.text = "Hello, test messages!!"
    
    context.send_activity = MagicMock(side_effect=mock_send_activity)

    try:
        # Call the on_message() function
        await on_message(context, None)
    except Exception as e:
        print("Exception: ", e)
    

# Call the test function
await test_on_message()

> You can find the following code in `api.py`

If we want to host a bot server, we need to use a `Flask` module to host it and define the handler activity of default bot message endpoint as `/api/messages`.

In [ ]:
# Define the Flask API
api = Flask(__name__)
# Define the route for receiving messages
@api.route('/api/messages', methods=['POST'])
async def on_messages():
    print("in on_messages")
    activity = Activity().deserialize(request.json)

    auth_header = request.headers['Authorization'] if 'Authorization' in request.headers else ''
    response = await app.process_activity(activity, auth_header)
    
    if response:
        return jsonify(response.body), response.status
    return '', 200

> You can find the following code in `app.py`

Now we can define the main block of the bot server.

In [ ]:

# Run the Flask API
if __name__ == "__main__":
    try:
        api.run(host='localhost', port=config.port)
    except Exception as error:
        raise error